In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [14]:
URL = 'https://www.indeed.com/jobs?q=data+science+job&l=New+York,+NY&jt=fulltime'
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, 'html.parser')
#printing soup in a more structured tree format that makes for easier reading
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script src="//d3fw5vlhllyvee.cloudfront.net/s/bc08208/en_US.js" type="text/javascript">
  </script>
  <link href="//d3fw5vlhllyvee.cloudfront.net/s/105b986/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="https://rss.indeed.com/rss?q=data+science+job&amp;l=New+York%2C+NY&amp;jt=fulltime" rel="alternate" title="Data Science Job Jobs, Employment in New York, NY" type="application/rss+xml"/>
  <link href="/m/jobs?q=data+science+job&amp;l=New+York%2C+NY&amp;jt=fulltime" media="only screen and (max-width: 640px)" rel="alternate"/>
  <script type="text/javascript">
   if (typeof window['closureReadyCallbacks'] == 'undefined') {
window['closureReadyCallbacks'] = [];
}

function call_when_jsall_loaded(cb) {
if (window['closureReady']) {
cb();
} else {
window['closureReadyCallbacks'].push(cb);
}
}
  </script>
  <meta content="1" name="ppstriptst"/>
  <sc

In [15]:
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name= 'div', attrs={'class':'row'}):
        for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
            jobs.append(a['title'])
        return(jobs)
extract_job_title_from_result(soup)

['Data Science Manager']

In [5]:
def extract_company_from_result(soup): 
    companies = []
    for div in soup.find_all(name='div', attrs={'class':'row'}):
        company = div.find_all(name='span', attrs={'class':'company'})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)
 
extract_company_from_result(soup)

['Employment Opportunities at BuzzFeed',
 'Warner Music Group',
 'S&P Global',
 'Murmuration',
 'HOF Capital',
 'Altice',
 'AlphaSights Ltd.',
 'AllianceBernstein',
 'Galvanize',
 'Amazon.com Services LLC',
 'Spotify',
 'Essence',
 'Quantbot Technologies LP',
 'New York University',
 'Thrive Global']

In [6]:
def extract_location_from_result(soup): 
    locations = []
    spans = soup.findAll('span', attrs={'class': 'location'})
    for span in spans:
        locations.append(span.text)
    return(locations)
extract_location_from_result(soup)

['New York, NY',
 'New York, NY',
 'New York, NY 10041 (Financial District area)',
 'New York, NY 10005 (Financial District area)',
 'New York, NY 10013 (Tribeca area)',
 'Long Island City, NY',
 'New York, NY',
 'New York, NY 10020 (Midtown area)',
 'New York, NY 10038 (Financial District area)',
 'New York, NY',
 'New York, NY',
 'New York, NY',
 'New York, NY 10017 (Murray Hill area)',
 'New York, NY',
 'New York, NY']

In [7]:
print(soup)

<!DOCTYPE html>

<html dir="ltr" lang="en">
<head>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<script id="polyfill-script-bundle">/* Disable minification (remove `.min` from URL path) for more info */

(function(self, undefined) {function ArrayCreate(r){if(1/r==-Infinity&&(r=0),r>Math.pow(2,32)-1)throw new RangeError("Invalid array length");var n=[];return n.length=r,n}function Call(t,l){var n=arguments.length>2?arguments[2]:[];if(!1===IsCallable(t))throw new TypeError(Object.prototype.toString.call(t)+"is not a function.");return t.apply(l,n)}function Get(n,t){return n[t]}function HasOwnProperty(r,t){return Object.prototype.hasOwnProperty.call(r,t)}function HasProperty(n,r){return r in n}function IsArray(r){return"[object Array]"===Object.prototype.toString.call(r)}function IsCallable(n){return"function"==typeof n}function RequireObjectCoercible(e){if(null===e||e===undefined)throw TypeError();return e}function SameValueNonNumber(e,n){return e===n}function ToBo

In [9]:
def extract_summary_from_result(soup): 
    summaries = []
    spans = soup.findAll('div', attrs={'class': 'summary'})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)
extract_summary_from_result(soup)

['Masters students in the areas of computer science, data science, information studies, & related fields are preferred.',
 'The Data Science Associate role is an entry-level position that offers associates with a high-impact introduction to applied data science in the music industry.',
 'You’ll design and leverage distributed computing technologies, data schemas, and APIs to construct data science pipelines.',
 'Familiarity with data science libraries e.g. scikit learn is a plus.\nCollaborate with data managers, data engineers, and data scientists to develop the…',
 'Majoring in data science, statistics, or a related field of study.\nResearch and evaluate important questions that require data to accurately answer.',
 'Exposure to big data systems and methodologies on cloud infrastructure.\nA strong passion for empirical research and for answering hard questions with data.',
 'Have strong SQL skills and enjoy extracting and manipulating data.\nHave a good degree of autonomy, owning the 

In [10]:
max_results_per_city = 100
city_set = ['New+York','Chicago','San+Francisco', 'Austin', 'Seattle', 'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Washington+DC', 'Boulder']

columns = ['city', 'job_title', 'company_name', 'location', 'summary', 'salary']

sample_df = pd.DataFrame(columns = columns)

In [11]:
for city in city_set:
    for start in range(0, max_results_per_city, 10):
        page = requests.get('http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=' + str(city) + '&start=' + str(start))
        time.sleep(1)  #ensuring at least 1 second between page grabs
    soup = BeautifulSoup(page.text, 'lxml', from_encoding='utf-8')
    for div in soup.find_all(name='div', attrs={'class':'row'}): 
    #specifying row num for index of job posting in dataframe
        num = (len(sample_df) + 1) 
    #creating an empty list to hold the data for each posting
        job_post = [] 
    #append city name
        job_post.append(city) 
    #grabbing job title
        for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
            job_post.append(a['title']) 
    #grabbing company name
        company = div.find_all(name='span', attrs={'class':'company'}) 
        if len(company) > 0: 
            for b in company:
                job_post.append(b.text.strip()) 
        else: 
            sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
            for span in sec_try:
                job_post.append(span.text) 
    #grabbing location name
    c = div.findAll('span', attrs={'class': 'location'}) 
    for span in c: 
        job_post.append(span.text) 
    #grabbing summary text
    d = div.findAll('span', attrs={'class': 'summary'}) 
    for span in d:
        job_post.append(span.text.strip())  
    #appending list of job post info to dataframe at index num
    sample_df.loc[num] = job_post

#saving sample_df as a local csv file — define your own local path to save contents 
sample_df.to_csv('[filepath].csv', encoding='utf-8')

/opt/anaconda3/envs/NLP_2021/lib/python3.8/site-packages/bs4/__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


ValueError: cannot set a row with mismatched columns